In [1]:
# %load_ext autoreload
# %autoreload 2

In [1]:
import sys
from pathlib import Path
proj_path = Path('.').resolve()
sys.path.append(str(proj_path))

__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

from langchain_chroma import Chroma
import matplotlib.pyplot as plt
import seaborn as sns
from copy import deepcopy

import json
import pickle
from tqdm import tqdm
import numpy as np
import pandas as pd
from typing import Optional
from collections import defaultdict
from dotenv import load_dotenv, find_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.documents import Document
from langchain_core.runnables import RunnableSequence
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_core.prompts import PromptTemplate

_ = load_dotenv(find_dotenv())

from src.db_utils import get_schema_str, get_db_file
from src.pymodels import (
    DatabaseModel, 
    SpiderSample, 
    BirdSample, 
    BODescription,
    SQLResponse,
    KeywordExtractionResponse,
    GenTemplateResponse
)
from src.prompts import Prompts
from src.database import SqliteDatabase
from src.data_preprocess import (
    load_raw_data,
    process_all_tables,
    filter_samples_by_count_spider_bird,
    process_samples_bird,
    split_train_dev_test,
    save_samples_spider_bird,
    load_samples_spider_bird,
)

from src.parsing_sql import (
    Schema, extract_all, extract_aliases, _format_expression, STRING_TYPE, NUMERIC_TYPE
)
from src.eval_utils import get_complexity, result_eq, check_if_exists_orderby
from run_bo_sql import get_vector_store
from copy import deepcopy

from src.eval_utils import get_structural_score, get_all_structural_score, get_all_semantic_score, partial_matching_with_penalty
from run_evaluation import get_target_parsed_sql, get_prediction_parsed_sql
from run_bo_sql import _get_categories, _format_interval, get_retriever
from bert_score import score as bscore
from transformers import logging as tfloggings
tfloggings.set_verbosity_error()
import warnings

/home/simonjisu/code/BusinessObjects/.venv/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/home/simonjisu/code/BusinessObjects/.venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [2]:
# from langchain.globals import set_llm_cache
# from langchain_community.cache import SQLiteCache
# set_llm_cache(SQLiteCache(database_path=f"./cache/valid_bo_bird_dev.db"))

In [16]:
from pprint import pprint
def get_content(x: dict):
    return json.loads(x['kwargs']['message']['kwargs']['content'])
task = 'direct'  
# task = 'keyword_extraction'
db_id = 'movie_3'
prefix = 'x-'
file_name = f'{task}_{db_id}'
db = SqliteDatabase(f"./cache/{prefix}{file_name}.db")

# df = db.execute("SELECT * FROM full_llm_cache WHERE ROWID = (SELECT MAX(ROWID) FROM full_llm_cache);")
df = db.execute("SELECT ROWID, response FROM full_llm_cache;")
df.shape

(600, 2)

In [17]:
content = get_content(json.loads(df['response'].iloc[-3]))
pprint(content)

{'hint_used': False,
 'rationale': ['Identify the customer by name: We need to find the customer '
               "'Marcelia Goering' in the 'customer' table to get the "
               'customer_id.',
               'Join the necessary tables: To find the orders placed by this '
               "customer, we need to join the 'customer', 'cust_order', and "
               "'order_line' tables.",
               'Filter by year: We need to filter the orders that were placed '
               'in the year 2021 by checking the order_date in the '
               "'cust_order' table.",
               'Filter by shipping method: We also need to ensure that the '
               "shipping method used is 'Priority Shipping' by joining with "
               "the 'shipping_method' table."],
 'sql': 'SELECT COUNT(*) FROM cust_order AS co INNER JOIN customer AS c ON '
        'co.customer_id = c.customer_id INNER JOIN shipping_method AS sm ON '
        "co.shipping_method_id = sm.method_id WHERE c.fir

In [18]:
x = '''To return latitude and longitude along with station details, I will select the lat and long columns from the station table.
'''.strip()
df.loc[df['response'].str.contains(x)]

,rowid,response
599,601,"{""lc"": 1, ""type"": ""constructor"", ""id"": [""langc..."


In [19]:
# idxes = df['response'].apply(lambda x: True if get_content(json.loads(x)).get(' ') else False)
# idxes = df['response'].str.contains("google_entity_id_id")
# idxes = ','.join(df.loc[idxes, 'rowid'].astype(str).tolist())
# idxes
idxes = ','.join(df.loc[df['response'].str.contains(x)].loc[:, 'rowid'].astype(str).values.tolist())
idxes

'601'

In [20]:
file_name = f'{task}_{db_id}'
db = SqliteDatabase(f"./cache/{prefix}{file_name}.db")
db.start()
c = db.con.cursor()
c.execute('BEGIN TRANSACTION')

# remove the last row record
c.execute(f"""
DELETE FROM full_llm_cache
WHERE ROWID IN ({idxes});
""")
db.con.commit()
db.close()
# (SELECT MAX(ROWID) FROM full_llm_cache)
# JSONDecodeError
# ValidationError
# c.execute("""
# DELETE FROM full_llm_cache WHERE response LIKE '%JSONDecodeError%';
# """)

In [ ]:
from pprint import pprint
df = db.execute(
'''
SELECT * FROM full_llm_cache
WHERE prompt LIKE '%Which teams have had a player awarded the Purple Cap and another with the Orange Cap%'
''')
df

---

In [ ]:
file_p1 = 'no_bos' if not args.with_bos else 'with_bos'
file_p2 = 'test' if args.type else 'dev'

file_p1 = 'no_bos'
file_p2 = 'dev'
file_name = f'{file_p1}-{file_p2}.json'

with open(prediction_path / file_name, 'r') as file:
    all_results = json.load(file)
    
for r in all_results:
    r['sql'] = r.pop('sql_template')

with open(prediction_path / file_name, 'w') as file:
    json.dump(all_results, file, indent=4)

In [ ]:
from src.eval_utils import get_structural_score, get_all_structural_score, get_all_semantic_score, partial_matching_with_penalty
from run_evaluation import get_target_parsed_sql, get_prediction_parsed_sql
from run_bo_sql import _get_categories, _format_interval, get_retriever
from bert_score import score as bscore
from transformers import logging as tfloggings
tfloggings.set_verbosity_error()
import warnings

ds = 'spider'
task = 'zero_shot_hint'
typ = 'test'
scenario = 0
description_file = f'description.json' if ds == 'spider' else f'{ds}_description.json'
experiment_folder = proj_path / 'experiments' / ds
prediction_path = experiment_folder / 'predictions' / task
eval_path = experiment_folder / 'evals' / task

tables, *_ = load_raw_data(proj_path / 'data' / ds, load_test=False)
with (proj_path / 'data' / description_file).open() as f:
    all_descriptions = json.load(f)
tables = process_all_tables(tables, descriptions=all_descriptions)

In [ ]:
# bird_path = proj_path / 'data' / 'bird'
# tables, train_data, dev_data = load_raw_data(bird_path, load_test=False)

# with (proj_path / 'data' / 'bird_description.json').open() as f:
#     all_descriptions = json.load(f)

# bird_tables = process_all_tables(tables, descriptions=all_descriptions)
# train_samples = load_samples_spider_bird(proj_path / 'data' / 'bird_train.json')
# dev_samples = load_samples_spider_bird(proj_path / 'data' / 'bird_dev.json')
# test_samples = load_samples_spider_bird(proj_path / 'data' / 'bird_test.json')

In [ ]:
# file_post_fix = f'{ds}_{typ}' if scenario < 0 else f'{ds}_{typ}_{scenario}'
# final_file = f'final_{file_post_fix}.json'
# samples = load_samples_spider_bird(proj_path / 'data' / f'{ds}_{typ}.json')

# if not (prediction_path / final_file).exists():
#     all_results = []
#     paths = sorted(list(prediction_path.glob(f'{file_post_fix}_*.json')))
#     for p in paths:
#         with p.open() as f:
#             results = json.load(f)
            
#         for r in results:
#             r.pop('rationale')
#             r['db_id'] = p.stem.split('_', 3)[-1]

#             found = False
#             for s in samples:
#                 if r['sample_id'] == s.sample_id:
#                     found = True
#                     break
#             r['gold_sql'] = s.final.sql
#             assert found, r['sample_id']

#         all_results.extend(results)

In [ ]:
# with open(prediction_path / final_file, 'r') as f:
#     preds = json.load(f)

In [ ]:
# pred_parsed, _ = get_prediction_parsed_sql(preds, tables)

# Training a retrieval model

In [3]:
ds = 'bird'
typ = 'dev'
task = 'retrieval' # 'zero_shot', 'retrieval', 'valid_bo'
description_file = f'description.json' if ds == 'spider' else f'{ds}_description.json'
experiment_folder = proj_path / 'experiments' / ds

eval_path = experiment_folder / 'evals' / task
prediction_path = experiment_folder / 'predictions' / task

tables, *_ = load_raw_data(proj_path / 'data' / ds, load_test=False)
with (proj_path / 'data' / description_file).open() as f:
    all_descriptions = json.load(f)
tables = process_all_tables(tables, descriptions=all_descriptions)

samples = load_samples_spider_bird(proj_path / 'data' / f'{ds}_{typ}.json')

In [ ]:
# with open(experiment_folder / 'predictions' / 'create_bo' / f'final_{ds}_train_bo.json', 'r') as f:
#     train_bo = json.load(f)

# df = []
# for db_id, xs in train_bo.items():
#     for x in xs:
#         x['db_id'] = db_id
#         df.append(x)

# df = pd.DataFrame(df)
# cates = pd.qcut(df['gold_complexity'], q=5)
# df['gold_complexity_cates'] = cates
# df['gold_complexity_codes'] = cates.cat.codes
# df.to_csv(experiment_folder / 'predictions' / 'create_bo' / f'final_{ds}_train_bo.csv', index=False)
# df = pd.read_csv(experiment_folder / 'predictions' / 'create_bo' / f'final_{ds}_train_bo.csv')

# df_train = df.groupby('gold_complexity_codes').sample(frac=0.9, random_state=42)
# df_dev = df.drop(df_train.index)

# print(df_train['gold_complexity_codes'].value_counts().sort_index())
# print(df_dev['gold_complexity_codes'].value_counts().sort_index())

# df_train.to_csv(experiment_folder / 'predictions' / 'create_bo' / f'{ds}_{task}_trainset.csv', index=False)
# df_dev.to_csv(experiment_folder / 'predictions' / 'create_bo' / f'{ds}_{task}_devset.csv', index=False)

# Procedures

## task: valid_bo

1. (retrieve) retrieve and rerank for dev dataset to get relevant train BOs
2. (gen_template) generate sql template with dev dataset and retrieved BOs --> output: dev sql templates
3. (keyword_extraction) extract keywords from dev sql templates --> output: keywords
4. (search_value) search for the most helpful values for the dev sql templates --> output: columns with values
5. (fill_in) fill-in values with dev sql templates --> output: dev sql
6. (evaluate) evaluate the dev sql to get the most helpful train BOs --> output: BOs Pool

## task: inference_with_bo

1. (retrieve) retrieve and rerank for test dataset from BOs Pool
2. (gen_template) generate sql template with test dataset and retrieved BOs --> output: test sql templates
3. (keyword_extraction) extract keywords from dev sql templates --> output: keywords
4. (search_value) search for the most helpful values for the dev sql templates --> output: columns with values
5. (fill_in) fill-in values with test sql templates --> output: test sql
6. (evaluate) evaluate the test sql


### retrieve analysis

In [2]:
# TODO: 
# 1. modify args.type == 'dev' to evaluate on `n_bos_sample` 
# 2. run pipeline for `n_bos_sample` x `n_dev_sample` (implement langchain batch)
# 3. run test set
# 4. implement direct method and run (only with bo)

In [4]:
class ARGS():
    exp_name = 'pipeline_exp'
    ds = 'bird'
    type = 'dev'
    task = 'retrieve'
    with_bos = True
    
args = ARGS()

description_file = f'description.json' if args.ds == 'spider' else f'{args.ds}_description.json'
experiment_folder = proj_path / args.exp_name / args.ds

description_file = 'bird_description.json' if args.ds == 'bird' else 'description.json'
tables, *_ = load_raw_data(proj_path / 'data' / args.ds, load_test=False)
with (proj_path / 'data' / description_file).open() as f:
    all_descriptions = json.load(f)
tables = process_all_tables(tables, descriptions=all_descriptions)

samples = load_samples_spider_bird(proj_path / 'data' / f'{args.ds}_{args.type}.json')

In [5]:
tables.keys()

dict_keys(['european_football_1', 'sales_in_weather', 'craftbeer', 'soccer_2016', 'restaurant', 'movie', 'olympics', 'language_corpus', 'app_store', 'sales', 'video_games', 'image_and_language', 'software_company', 'authors', 'movies_4', 'social_media', 'human_resources', 'regional_sales', 'computer_student', 'works_cycles', 'food_inspection_2', 'citeseer', 'bike_share_1', 'law_episode', 'cs_semester', 'legislator', 'world', 'cookbook', 'university', 'books', 'shipping', 'food_inspection', 'movie_platform', 'shakespeare', 'book_publishing_company', 'car_retails', 'mental_health_survey', 'hockey', 'music_platform_2', 'address', 'menu', 'professional_basketball', 'cars', 'synthea', 'genes', 'retails', 'talkingdata', 'beer_factory', 'chicago_crime', 'mondial_geo', 'student_loan', 'codebase_comments', 'retail_world', 'music_tracker', 'disney', 'college_completion', 'ice_hockey_draft', 'world_development_indicators', 'airline', 'retail_complains', 'trains', 'public_review_platform', 'donor'

In [6]:
db_id = 'language_corpus'

db_file = get_db_file(proj_path, 'bird', db_id)
db = SqliteDatabase(db_file)

In [7]:
db.execute("SELECT * FROM sqlite_master WHERE type='table';")

,type,name,tbl_name,rootpage,sql
0,table,langs,langs,2,CREATE TABLE langs(lid INTEGER PRIMARY KEY ...
1,table,sqlite_sequence,sqlite_sequence,5,"CREATE TABLE sqlite_sequence(name,seq)"
2,table,pages,pages,6,CREATE TABLE pages(pid INTEGER PRIMARY KEY AUT...
3,table,words,words,8,CREATE TABLE words(wid INTEGER PRIMARY KEY AUT...
4,table,langs_words,langs_words,10,CREATE TABLE langs_words(lid INTEGER REFERENCE...
5,table,pages_words,pages_words,11,CREATE TABLE pages_words(pid INTEGER REFERENCE...
6,table,biwords,biwords,12,CREATE TABLE biwords(lid INTEGER REFERENCES...


In [10]:
x = db.execute("SELECT * FROM words;", rt_pandas=False)

In [12]:
len(x)

2764996

In [ ]:
(x[:200000])

[(1, 'àbac', 242),
 (2, 'xinès', 16841),
 (3, 'llatí', 48700),
 (4, 'grec', 49897),
 (5, 'antic', 60220),
 (6, 'càlcul', 7039),
 (7, 'aritmètica', 1841),
 (8, 'nombre', 115938),
 (9, 'enter', 2174),
 (10, 'decimal', 1522),
 (11, 'base', 60713),
 (12, 'u', 25416),
 (13, 'desena', 2593),
 (14, 'suma', 7802),
 (15, 'resta', 48686),
 (16, 'multiplicació', 1895),
 (17, 'divisió', 71303),
 (18, 'gerbert', 201),
 (19, "d'orlhac", 267),
 (20, 'vic', 14000),
 (21, 'ripoll', 5749),
 (22, 'borrell', 3055),
 (23, 'ii', 164838),
 (24, 'barcelona', 272289),
 (25, 'neperià', 56),
 (26, 'john', 81745),
 (27, 'napier', 597),
 (28, 'amèrica', 47892),
 (29, 'precolombina', 722),
 (30, 'yupana', 18),
 (31, 'quipu', 61),
 (32, 'espanya', 90071),
 (33, 'nombres', 13860),
 (34, 'de', 19877053),
 (35, 'fibonacci', 483),
 (36, 'antiga', 32624),
 (37, 'xina', 50675),
 (38, 'japó', 49430),
 (39, 'sumer', 772),
 (40, 'sistema', 120714),
 (41, 'sexagesimal', 577),
 (42, 'heròdot', 2705),
 (43, 'egipte', 22327),
 (

In [ ]:
# Sample code for dev
from itertools import product

retrieval_path: Path = experiment_folder / 'predictions' / 'retrieve' / f'with_bos-{args.type}.json'
with retrieval_path.open() as f:
    retrieved = json.load(f)

df = {'db_id': [], 'ss': [], 'bo': []}
retrieved_by_db_ids = defaultdict(set)
for r in retrieved:
    db_id = r['db_id']
    sample_id = r['sample_id']
    ret = r['retrieved']
    for s, bo in product([sample_id], ret):
        df['db_id'].append(db_id)
        df['ss'].append(s)
        df['bo'].append(bo)
    retrieved_by_db_ids[db_id].update(ret)
df = pd.DataFrame(df)

n_bos_aug = 30
sampled_bos = defaultdict(list)
for db_id, bos in retrieved_by_db_ids.items():
    if len(bos) > n_bos_aug:
        n_sample = n_bos_aug
    else:
        # sample half of the bos
        n_sample = len(bos) // 2
    sampled = np.random.choice(list(bos), n_sample, replace=False).tolist()
    sampled_bos[db_id].extend(sampled)

# with (experiment_folder / 'predictions' / 'retrieve' / f'with_bos-sampled.json').open('w') as f:
#     json.dump(sampled_bos, f, indent=4)

In [3]:
def remove_duplicate_bos(
        bos: dict[str, list[dict[str, str]]]
    ) -> dict[str, list[dict[str, str]]]:
    hash_map = defaultdict(set)
    for db_id, xs in bos.items():
        for x in xs:
            hash_map[x['vt']].add(x['sample_id'])

    ids = list(map(lambda x: list(x)[0], hash_map.values()))
    new_bos = {}
    for db_id, xs in bos.items():
        new_bos[db_id] = [x for x in xs if x['sample_id'] in ids]

    return new_bos

with (experiment_folder / 'predictions' / 'retrieve' / f'with_bos-sampled.json').open('r') as f:
    retrieved = json.load(f)

bo_path: Path = experiment_folder / 'predictions' / 'create_bo' / f'final_{args.ds}_train_bo.json'
with bo_path.open() as f:
    bos = json.load(f)
bos = remove_duplicate_bos(bos)


if args.with_bos:
    prompt_template = Prompts.gen_template_with_bos
    input_variables = ['schema', 'input_query', 'hint']
    structured_output = GenTemplateResponse
else:
    prompt_template = Prompts.gen_template_no_bos
    input_variables = ['schema', 'input_query']
    structured_output = SQLResponse

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=input_variables,
)

model_openai = ChatOpenAI(
    model='gpt-4o-mini',
    temperature=0.0,
    frequency_penalty=0.1,
)
model = model_openai.with_structured_output(structured_output, method='json_mode')
chain = (prompt | model)

In [4]:
from itertools import product, islice
def batched(iterable, n, *, strict=False):
    # batched('ABCDEFG', 3) → ABC DEF G
    if n < 1:
        raise ValueError('n must be at least one')
    iterator = iter(iterable)
    while batch := tuple(islice(iterator, n)):
        if strict and len(batch) != n:
            raise ValueError('batched(): incomplete batch')
        yield batch

In [5]:
from langchain_community.callbacks.manager import get_openai_callback

In [106]:
def create_hint(id2bo, doc_ids):
    docs = [{
        'descrption': id2bo[doc_id]['ba'], 
        'virtual_table': id2bo[doc_id]['vt']
    } for doc_id in doc_ids]
    
    hint = '\nDescriptions and Virtual Tables:\n'
    hint += json.dumps({j: doc for j, doc in enumerate(docs)}, indent=4)
    hint += '\n'
    return hint

with_bos = args.with_bos
is_test = False
if with_bos and is_test:
    retrieved_by_db_id = defaultdict(dict)
    for sample in retrieved:
        db_id = sample['db_id']
        sample_id = sample['sample_id']
        retrieved_by_db_id[db_id][sample_id] = sample['retrieved']

samples_by_db_id = defaultdict(list)
for sample in samples:
    samples_by_db_id[sample.db_id].append(sample)


for db_id, samples in samples_by_db_id.items():    
    id2bo = {} 
    schema_str = get_schema_str(
        schema=tables[db_id].db_schema, 
        foreign_keys=tables[db_id].foreign_keys,
        col_explanation=tables[db_id].col_explanation
    )
    for x in bos[db_id]:
        id2bo[x['sample_id']] = x

    n_batch = 32 if is_test else 1
    results = []
    for batch in batched(samples, n_batch):
        batch_inputs = []
        batch_retrieved = []
        batch_sample_ids = []
        if with_bos:
            if is_test:
                for sample in batch:
                    question = sample.final.question
                    doc_ids = retrieved_by_db_id[db_id][sample.sample_id]
                    hint = create_hint(id2bo, doc_ids)
                    input_data = {'schema': schema_str, 'input_query': question, 'hint': hint}
                    batch_inputs.append(input_data)
                    batch_retrieved.append(doc_ids)
                    batch_sample_ids.append(sample.sample_id)
            else:
                for bo_id, sample in product(retrieved[db_id], batch):
                    question = sample.final.question
                    doc_ids = [bo_id]
                    hint = create_hint(id2bo, doc_ids)
                    input_data = {'schema': schema_str, 'input_query': question, 'hint': hint}
                    batch_inputs.append(input_data)
                    batch_retrieved.append(doc_ids)
                    batch_sample_ids.append(sample.sample_id)
        else:
            for sample in batch:
                question = sample.final.question
                input_data = {'schema': schema_str, 'input_query': question}
                batch_inputs.append(input_data)
                batch_retrieved.append([])
                batch_sample_ids.append(sample.sample_id)
        
        # with get_openai_callback() as cb:
        #     batch_outputs: list[GenTemplateResponse] = chain.batch(inputs=batch_inputs)

        # for sample, output in zip(batch, batch_outputs):
        #     res = {}
        #     res['sample_id'] = sample.sample_id
        #     res['rationale'] = output.rationale
        #     res['sql_template'] = output.sql
        #     if with_bos:
        #         res['hint_used'] = output.hint_used
        #     res['token_usage'] = {'tokens': cb.total_tokens/len(batch), 'cost': cb.total_cost/len(batch)}
        #     results.append(res)
        break
    break

In [14]:
with get_openai_callback() as cb:
    batch_outputs: list[GenTemplateResponse] = chain.batch(inputs=batch_inputs)

### keyword extraction

In [193]:
from langchain.globals import set_llm_cache
from langchain_community.cache import SQLiteCache

In [195]:
class ARGS():
    exp_name = 'pipeline_exp'
    ds = 'bird'
    type = 'dev'
    task = 'keyword_extraction'
    with_bos = True
    
args = ARGS()

description_file = f'description.json' if args.ds == 'spider' else f'{args.ds}_description.json'
experiment_folder = proj_path / args.exp_name / args.ds

description_file = 'bird_description.json' if args.ds == 'bird' else 'description.json'
tables, *_ = load_raw_data(proj_path / 'data' / args.ds, load_test=False)
with (proj_path / 'data' / description_file).open() as f:
    all_descriptions = json.load(f)
tables = process_all_tables(tables, descriptions=all_descriptions)

samples = load_samples_spider_bird(proj_path / 'data' / f'{args.ds}_{args.type}.json')

prediction_path = experiment_folder / 'predictions' / args.task


In [177]:
file_name = f'{"with_bos" if args.with_bos else "no_bos"}-{args.type}'
template_path = experiment_folder / 'predictions' / 'gen_template' / f'{file_name}.json'
with template_path.open() as f:
    templates = json.load(f)
samples_by_id = {s.sample_id: s for s in samples}

In [185]:
# same template will have one keyword extraction inference
input_samples = []
template_sample_id2doc_ids = defaultdict(set)  # template, sample_id: {doc_id}
for t in templates:
    sample_id: int = t['sample_id']
    doc_ids: list = t['doc_ids']   # [] for no_bos
    sql_template: str = t['sql_template']
    key = (sql_template, sample_id)
    if key not in template_sample_id2doc_ids:
        sample = samples_by_id[sample_id]
        x = {
            'sample_id': sample_id,
            'question': sample.final.question,
            'evidence': sample.evidence,
            'db_id': sample.db_id,
            'sql_template': sql_template,
        }
        input_samples.append(x)
        template_sample_id2doc_ids[key].update(doc_ids)

    for doc_id in doc_ids:
        template_sample_id2doc_ids[key].add(doc_id)

if not (args.with_bos and args.type == 'dev'):
    # dev(with_bos): sample_id + doc_ids (need to evaluate for doc_ids)
    # dev(no_bos): sample_id
    # test(both cases): sample_id (under test retrieval number = 1)
    assert len(template_sample_id2doc_ids) == len(templates), f"{len(template_sample_id2doc_ids)} != {len(templates)}"

prompt = PromptTemplate(
    template=Prompts.keyword_extraction,
    input_variables=['schema', 'input_query', 'evidence', 'sql_template'],
)
model_openai = ChatOpenAI(
    model='gpt-4o-mini',
    temperature=0.5,
    frequency_penalty=0.1,
)
model = model_openai.with_structured_output(KeywordExtractionResponse, method='json_mode')
chain = (prompt | model)

In [183]:
len(input_samples), len(templates)

(29368, 57902)

In [188]:
samples_by_db_id = defaultdict(list)
for sample in input_samples:
    # sample: {sample_id, db_id, question, evidence, sql_template}
    db_id = sample['db_id']
    if db_id not in []:
        x = {
            'sample_id': sample['sample_id'],
            'question': sample['question'],
            'evidence': sample['evidence'],
            'sql_template': sample['sql_template'],
        }
        samples_by_db_id[db_id].append(x)

In [202]:
db_id = 'legislator'
samples = samples_by_db_id[db_id]
set_llm_cache(SQLiteCache(database_path=f"./cache/{prefix}{prediction_path.stem}_{db_id}.db"))
results = []
schema_str = get_schema_str(schema=tables[db_id].db_schema)
n_batch = 32
batched_samples: list[dict] = list(batched(samples, n_batch))
batch = batched_samples[0]

In [203]:
batch_inputs = []
batch_iidx2oidx: dict[int, int] = {}  # input_idx --> output_idx for batch_outputs

for iidx, sample in enumerate(batch):
    # STRING_TYPE = '[PLACEHOLDER-TYPE:STRING]'.lower()
    # NUMERIC_TYPE = '[PLACEHOLDER-TYPE:NUMERIC]'.lower()
    # only need to inference on the queries that has placeholders
    if (STRING_TYPE.lower() in sample['sql_template'].lower()) or (NUMERIC_TYPE.lower() in sample['sql_template'].lower()):
        input_data = {
            'schema': schema_str,
            'input_query': sample['question'],
            'evidence': sample['evidence'],
            'sql_template': sample['sql_template'] 
        }
        batch_inputs.append(input_data)
        batch_iidx2oidx[iidx] = len(batch_inputs) - 1

if batch_inputs:
    with get_openai_callback() as cb:
        batch_outputs: list[KeywordExtractionResponse] = chain.batch(inputs=batch_inputs)
else:
    batch_outputs = []

In [220]:
results = []
for iidx, sample in enumerate(batch):
    sample_id = sample['sample_id']
    sql_template = sample['sql_template']
    res = {'sample_id': sample_id}
    
    oidx = batch_iidx2oidx.get(iidx)
    if oidx is not None:
        res['keywords'] = batch_outputs[oidx].extraction
    else:
        res['keywords'] = {}
    res['token_usage'] = {'tokens': cb.total_tokens/n_batch, 'cost': cb.total_cost/n_batch}
    key = (sql_template, sample_id)
    doc_ids = template_sample_id2doc_ids.get(key)
    if doc_ids:
        for doc_id in doc_ids:
            new_res = deepcopy(res)
            new_res['doc_ids'] = [doc_id]
            results.append(new_res)
    else:
        res['doc_ids'] = []
        results.append(res)
    break

### search values

In [134]:
class ARGS():
    exp_name = 'pipeline_exp'
    ds = 'bird'
    type = 'dev'
    task = 'search_value'
    with_bos = True
    
args = ARGS()

description_file = f'description.json' if args.ds == 'spider' else f'{args.ds}_description.json'
experiment_folder = proj_path / args.exp_name / args.ds

description_file = 'bird_description.json' if args.ds == 'bird' else 'description.json'
tables, *_ = load_raw_data(proj_path / 'data' / args.ds, load_test=False)
with (proj_path / 'data' / description_file).open() as f:
    all_descriptions = json.load(f)
tables = process_all_tables(tables, descriptions=all_descriptions)

samples = load_samples_spider_bird(proj_path / 'data' / f'{args.ds}_{args.type}.json')

def _format_column_value(string: str):
    operations = [
        '>', '<', '=', '>=', '<=', '!=', '<>'
    ]
    found = False
    for op in operations:
        if op in string.lower():
            found = True
            break
    
    if found:
        # split the string by the operator
        string = string.split(op, 1)[-1].strip()
    
    # remove ' or " from the string
    string = string.replace("'", '').replace('"', '')
    return string

samples = load_samples_spider_bird(proj_path / 'data' / f'{args.ds}_{args.type}.json')

file_name = f'{"with_bos" if args.with_bos else "no_bos"}-{args.type}'
# keyword_path = experiment_folder / 'predictions' / 'keyword_extraction' / f'{file_name}.json'
keyword_path = experiment_folder / 'predictions' / 'keyword_extraction' / f'with_bos-dev2.json'
assert keyword_path.exists(), f'Run with the `task=keyword_extraction, type={args.type}` first'
with keyword_path.open() as f:
    keywords = json.load(f)

In [135]:
samples_by_id = {s.sample_id: s for s in samples}

input_samples = []
keyword_sample_id2doc_ids = defaultdict(set) # (all_kws, sample_id): {doc_id}
for k in keywords:
    sample_id: int = k['sample_id']
    doc_ids: list = k['doc_ids']
    # process keywords
    output_kws: dict[str, list[str]] = k['keywords']
    # sort keywords first
    output_kws = {k: sorted(v) for k, v in output_kws.items()}
    all_kws: dict[str, list[str]] = {}
    for output_column_name, kws in output_kws.items():
        if kws:  # only store non-empty keywords
            if ('.' in output_column_name): # rewrite the column_name
                _, column_name = output_column_name.split('.')
            else:
                column_name = output_column_name
            all_kws[column_name] = [_format_column_value(kw) for kw in kws if kw]

    key = (json.dumps(all_kws), sample_id)
    if key not in keyword_sample_id2doc_ids:
        sample = samples_by_id[sample_id]
        x = {
            'sample_id': sample_id,
            'db_id': sample.db_id,
            'db_file': get_db_file(proj_path, args.ds, sample.db_id),
            'search_keywords': all_kws
        }
        input_samples.append(x)
        keyword_sample_id2doc_ids[key].update(doc_ids)

In [137]:
len(input_samples), len(keywords)

(208, 680)

### fill_in

In [ ]:
class ARGS():
    exp_name = 'pipeline_exp'
    ds = 'bird'
    type = 'dev'
    task = 'fill_in'
    with_bos = True
    
args = ARGS()

description_file = f'description.json' if args.ds == 'spider' else f'{args.ds}_description.json'
experiment_folder = proj_path / args.exp_name / args.ds

description_file = 'bird_description.json' if args.ds == 'bird' else 'description.json'
tables, *_ = load_raw_data(proj_path / 'data' / args.ds, load_test=False)
with (proj_path / 'data' / description_file).open() as f:
    all_descriptions = json.load(f)
tables = process_all_tables(tables, descriptions=all_descriptions)

samples = load_samples_spider_bird(proj_path / 'data' / f'{args.ds}_{args.type}.json')

file_name = f'{"with_bos" if args.with_bos else "no_bos"}-{args.type}'
values_path = experiment_folder / 'predictions' / 'search_value' / f'{file_name}.json'
assert values_path.exists(), f'Run with the `task=search_value, type={args.type}` first'
with values_path.open() as f:
    searched_values = json.load(f)

template_path = experiment_folder / 'predictions' / 'gen_template' / f'{file_name}.json'
assert template_path.exists(), f'Run with the `task=gen_template, type={args.type}` first'
with template_path.open() as f:
    templates = json.load(f)

In [ ]:
# prepare inputs
samples_by_id = {s.sample_id: s for s in samples}
values_by_id = {v['sample_id']: v for v in searched_values}
# same template will have one keyword extraction inference
input_samples = []
template_values_sample_id2doc_ids = defaultdict(set)  # template, sample_id: {doc_id}
for t in templates:
    sample_id: int = t['sample_id']
    doc_ids: list = t['doc_ids']   # [] for no_bos
    sql_template: str = t['sql_template']
    values = values_by_id[sample_id]['values'] if values_by_id.get(sample_id) else {}
    # sort values
    if values:
        values = {
            table_name: {column_name: sorted(vs) for column_name, vs in column_values.items()}
                   for table_name, column_values in values.items()}
    key = (sql_template, json.dumps(values), sample_id)
    if key not in template_values_sample_id2doc_ids:
        sample = samples_by_id[sample_id]
        x = {
            'sample_id': sample_id,
            'db_id': sample.db_id,
            'question': sample.final.question,
            'values': values,
            'sql_template': sql_template,
        }
        input_samples.append(x)
        template_values_sample_id2doc_ids[key].update(doc_ids)

    for doc_id in doc_ids:
        template_values_sample_id2doc_ids[key].add(doc_id)

if not (args.with_bos and args.type == 'dev'):
    # dev(with_bos): sample_id + doc_ids (need to evaluate for doc_ids)
    # dev(no_bos): sample_id
    # test(both cases): sample_id (under test retrieval number = 1)
    assert len(template_values_sample_id2doc_ids) == len(templates), f"{len(template_values_sample_id2doc_ids)} != {len(templates)}"


In [166]:
len(input_samples), len(templates)

(29368, 57902)

### evaluate

In [2]:
class ARGS():
    exp_name = 'direct_exp'
    ds = 'bird'
    type = 'dev'
    task = 'evaluate'
    eval_target = 'direct'
    with_bos = False
    
args = ARGS()

description_file = f'description.json' if args.ds == 'spider' else f'{args.ds}_description.json'
experiment_folder = proj_path / args.exp_name / args.ds
eval_path = experiment_folder / 'evals'

description_file = 'bird_description.json' if args.ds == 'bird' else 'description.json'
tables, *_ = load_raw_data(proj_path / 'data' / args.ds, load_test=False)
with (proj_path / 'data' / description_file).open() as f:
    all_descriptions = json.load(f)
tables = process_all_tables(tables, descriptions=all_descriptions)


def parse_sql_to_output(sql: str, schema: Schema):
    try:
        ei = extract_all(sql, schema)
        assert len(ei['sel']) > 0, f'No selection found'
    except Exception as e:
        return None
    return ei

samples = load_samples_spider_bird(proj_path / 'data' / f'{args.ds}_{args.type}.json')

file_name = f'{"with_bos" if args.with_bos else "no_bos"}-{args.type}'
eval_target_path = experiment_folder / 'predictions' / args.eval_target / f'{file_name}.json'
assert eval_target_path.exists(), f'Run with the `task={args.eval_target}, type={args.type}` first'
with eval_target_path.open() as f:
    eval_targets = json.load(f)

# TODO: for loop should be start from pred_queries

In [4]:
import hashlib

In [5]:
parsed_queries_path: Path = eval_path / f'parsed_queries-{file_name}.pkl'
parsed_queries_exist = parsed_queries_path.exists()
if parsed_queries_exist:
    with parsed_queries_path.open('rb') as f:
        parsed_queries = pickle.load(f)
else:
    parsed_queries = defaultdict(dict)
    parsed_queries['unable'] = set()

samples_by_id = {s.sample_id: s for s in samples}
eval_data: dict[str, list] = defaultdict(list)
eval_data2doc_ids = defaultdict(set)

# from flatten to nested by doc_ids
for pred in tqdm(eval_targets, total=len(eval_targets), desc='preparing eval data'):
    sample_id = pred['sample_id']
    pred_sql = pred['sql']
    doc_ids = pred['doc_ids']
    target_sample = samples_by_id[sample_id]
    target_sql = target_sample.final.sql
    db_id = target_sample.db_id
    key = hashlib.sha256(f'{sample_id}-{pred_sql}'.encode()).hexdigest()

    if not parsed_queries_exist:
        schema = Schema(tables[db_id].db_schema)
        if parsed_queries['target'].get(target_sql) is None:
            target_output = parse_sql_to_output(target_sql, schema)
            parsed_queries['target'][target_sql] = target_output
        else:
            target_output = parsed_queries['target'][target_sql]
        if parsed_queries['pred'].get(pred_sql) is None:
            pred_output = parse_sql_to_output(pred_sql, schema)
            parsed_queries['pred'][pred_sql] = pred_output
        else:
            pred_output = parsed_queries['pred'][pred_sql]
    
        if (not pred_output) or (not target_output):
            if key not in parsed_queries['unable']:
                parsed_queries['unable'].add(key)
            continue
    else:
        if key in parsed_queries['unable']:
            continue
    
    if key not in eval_data2doc_ids:
        eval_data['sample_ids'].append(sample_id)
        eval_data['target_queries'].append(target_sql)
        eval_data['db_paths'].append(get_db_file(proj_path, args.ds, db_id))
        eval_data['pred_queries'].append(pred_sql)
        eval_data2doc_ids[key].update(doc_ids)

    for doc_id in doc_ids:
        eval_data2doc_ids[key].add(doc_id)

# if not parsed_queries_exist:
#     with parsed_queries_path.open('wb') as f:
#         pickle.dump(parsed_queries, f)

preparing eval data: 100%|██████████| 2091/2091 [00:00<00:00, 10097.80it/s]


In [34]:
from itertools import product, islice
def batched(iterable, n, *, strict=False):
    # batched('ABCDEFG', 3) → ABC DEF G
    if n < 1:
        raise ValueError('n must be at least one')
    iterator = iter(iterable)
    while batch := tuple(islice(iterator, n)):
        if strict and len(batch) != n:
            raise ValueError('batched(): incomplete batch')
        yield batch

n_batch = 200
n_samples = len(eval_data['sample_ids'])
batches = list(batched(range(n_samples), n_batch))

In [35]:
for idxes in batches:
    print(len(idxes), len(batches))
    batch_eval_data = {k: [v[i] for i in idxes] for k, v in eval_data.items()}
    print(len(batch_eval_data['sample_ids']), idxes[:5])

200 11
200 (0, 1, 2, 3, 4)
200 11
200 (200, 201, 202, 203, 204)
200 11
200 (400, 401, 402, 403, 404)
200 11
200 (600, 601, 602, 603, 604)
200 11
200 (800, 801, 802, 803, 804)
200 11
200 (1000, 1001, 1002, 1003, 1004)
200 11
200 (1200, 1201, 1202, 1203, 1204)
200 11
200 (1400, 1401, 1402, 1403, 1404)
200 11
200 (1600, 1601, 1602, 1603, 1604)
200 11
200 (1800, 1801, 1802, 1803, 1804)
29 11
29 (2000, 2001, 2002, 2003, 2004)


In [14]:
len(batch_eval_data['sample_ids'])

200

### aggregate

In [ ]:
file_name = f'{"with_bos" if args.with_bos else "no_bos"}-{args.type}'

samples = load_samples_spider_bird(proj_path / 'data' / f'{args.ds}_{args.type}.json')
db_ids = {s.sample_id: s.db_id for s in samples}

if args.with_bos:  # whether to use BOs or not
    with open(experiment_folder / 'predictions' / 'retrieve' / f'{file_name}.json', 'r') as f:
        retrieved = json.load(f)
    retrieved = {r['sample_id']: r for r in retrieved}
else:
    retrieved = {}

if args.eval_target == 'fill_in':  # pipeline method
    # gen_template
    with open(experiment_folder / 'predictions' / 'gen_template' / f'{file_name}.json', 'r') as f:
        sql_templates = json.load(f)
    sql_templates = {r['sample_id']: r for r in sql_templates}

    # keyword_extraction
    with open(experiment_folder / 'predictions' / 'keyword_extraction' / f'{file_name}.json', 'r') as f:
        keyword_extraction = json.load(f)
    keyword_extraction = {r['sample_id']: r for r in keyword_extraction}

    # search_value
    with open(experiment_folder / 'predictions' / 'search_value' / f'{file_name}.json', 'r') as f:
        search_value = json.load(f)
    search_value = {r['sample_id']: r for r in search_value}
else:
    sql_templates = {}
    keyword_extraction = {}
    search_value = {}

with open(experiment_folder / 'evals' / f'execution_result-{file_name}.json', 'r') as f:
    exec_results = json.load(f)
exec_results = {r['sample_id']: r for r in exec_results}

with open(experiment_folder / 'evals' / f'merit_result-{file_name}.json', 'r') as f:
    merit_results = json.load(f)
merit_results = {r['sample_id']: r for r in merit_results}

all_results = defaultdict(list)

for sample_id in merit_results.keys():
    all_results['sample_id'].append(sample_id)
    all_results['db_id'].append(db_ids[sample_id])
    if args.with_bos:
        bo_ids = ','.join(map(str, retrieved[sample_id]['retrieved']))
        bo_used = int(sql_templates[sample_id]['hint_used'])
        all_results['bo_ids'].append(bo_ids)
        all_results['bo_used'].append(bo_used)

    if args.eval_target == 'fill_in':
        keywords = json.dumps({column_name: kws for column_name, kws in keyword_extraction[sample_id]['keywords'].items() if kws})
        # keywords = False if keywords == '{}' else True
        values = json.dumps(search_value[sample_id]['values'])
        # values = False if values == '{}' else True
        all_results['keywords'].append(keywords)
        all_results['values'].append(values)
    all_results['exec_res'].append(exec_results[sample_id]['res'])
    all_results['structural_score'].append(merit_results[sample_id]['structural_score'])
    all_results['semantic_score'].append(merit_results[sample_id]['semantic_score'])
    all_results['f1_score'].append(merit_results[sample_id]['f1_score'])
    all_results['target_complexity'].append(merit_results[sample_id]['target_complexity'])

### valid_bo

In [224]:
class ARGS():
    exp_name = 'pipeline_exp'
    ds = 'bird'
    type = 'dev'
    task = 'valid_bo'
    with_bos = False
    
args = ARGS()

description_file = f'description.json' if args.ds == 'spider' else f'{args.ds}_description.json'
experiment_folder = proj_path / args.exp_name / args.ds

description_file = 'bird_description.json' if args.ds == 'bird' else 'description.json'
tables, *_ = load_raw_data(proj_path / 'data' / args.ds, load_test=False)
with (proj_path / 'data' / description_file).open() as f:
    all_descriptions = json.load(f)
tables = process_all_tables(tables, descriptions=all_descriptions)

samples = load_samples_spider_bird(proj_path / 'data' / f'{args.ds}_{args.type}.json')

In [225]:
eval_path = experiment_folder / 'evals'

df_base = pd.read_csv(eval_path / f'result-with_bos-{args.type}.csv')
df_base['f1_score'] = np.random.rand(df_base.shape[0])
df_base.drop(columns=['bo_ids', 'bo_used', 'keywords', 'values'], inplace=True)
df_bo = pd.read_csv(eval_path / f'result-with_bos-{args.type}.csv')
# df_no_bo = pd.read_csv(eval_path / f'result-no_bos-{args.type}.csv')

In [229]:
df_cates = df_base.groupby('db_id')['target_complexity'].apply(_get_categories).rename('category').apply(_format_interval)
df_base = pd.merge(df_base, df_cates.reset_index('db_id', drop=True), left_index=True, right_index=True)

In [176]:
df = pd.merge(
    left=df_bo,
    right=df_base,
    how='inner',
    on=['db_id', 'sample_id', 'target_complexity'],
    suffixes=('_bo', '_base')
)

In [177]:
df.sort_values(by='sample_id')

,sample_id,db_id,bo_ids,bo_used,keywords,values,exec_res_bo,structural_score_bo,semantic_score_bo,f1_score_bo,target_complexity,exec_res_base,structural_score_base,semantic_score_base,f1_score_base,category
6,3,movie_platform,45,0,{},{},0,0.5657,0.8888,0.691364,9,0,0.5657,0.8888,0.357111,"(8, 9]"
4,5,movie_platform,82,1,"{""movie_title"": [""When Will I Be Loved""]}","{""movies"": {""movie_title"": [""When Will I Be Lo...",1,1.0000,1.0000,1.000000,7,1,1.0000,1.0000,0.519125,"(4, 7]"
14,7,movie_platform,136,1,{},{},0,0.4828,0.9713,0.644995,5,0,0.4828,0.9713,0.980458,"(4, 7]"
24,8,movie_platform,100,1,"{""rating_date_utc"": [""2020-04-01"", ""2020-04-30...","{""ratings_users"": {""rating_date_utc"": [""2020-0...",0,0.6489,0.9158,0.759587,9,0,0.6489,0.9158,0.710273,"(8, 9]"
27,13,movie_platform,82,1,"{""movie_title"": [""Welcome to the Dollhouse""]}","{""movies"": {""movie_title"": [""Welcome to the Do...",1,1.0000,1.0000,1.000000,8,1,1.0000,1.0000,0.534417,"(7, 8]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2002,10924,debit_card_specializing,10923,1,"{""currency"": [""euro"", ""EUR""]}","{""products"": {""description"": [""Eurovignette""]}...",0,0.5138,0.7049,0.594367,9,0,0.5138,0.7049,0.246214,"(7, 9]"
2001,10929,debit_card_specializing,10936,1,"{""chainid"": [""11""]}","{""gasstations"": {""chainid"": [""11""]}, ""transact...",0,0.8738,0.9658,0.917499,8,0,0.8738,0.9658,0.715064,"(7, 9]"
1998,10930,debit_card_specializing,10931,1,"{""country"": [""CZE""], ""price"": [""1000""]}","{""gasstations"": {""country"": [""CZE""]}}",1,0.9333,0.9639,0.948353,9,1,0.9333,0.9639,0.331370,"(7, 9]"
1999,10939,debit_card_specializing,10918,1,{},{},1,0.7083,0.7175,0.712870,8,1,0.7083,0.7175,0.625524,"(7, 9]"


# Similarity between dataset

In [ ]:
def get_parsed_sql(samples: dict, tables: dict):
    error_ids = []
    parsed = defaultdict(dict)
    iterator = tqdm(samples, total=len(samples))
    for sample in iterator:
        db_id = sample.db_id
        sample_id = sample.sample_id
        iterator.set_description(f"{db_id}")
        schema = Schema(tables[db_id].db_schema)
        sql_i = sample.final.sql
        try:
            ei = extract_all(sql_i, schema)
            assert len(ei['sel']) > 0, f'No selection found-{db_id}-{sample_id}'
        except Exception as e:
            error_ids.append((db_id, sample_id, str(e)))
            parsed[db_id].append(None)
            continue
        parsed[db_id][sample_id] = ei
    return parsed, error_ids

train_parsed, error_ids = get_parsed_sql(train_samples, bird_tables)
dev_parsed, error_ids = get_parsed_sql(dev_samples, bird_tables)
test_parsed, error_ids = get_parsed_sql(test_samples, bird_tables)

In [ ]:
# with open(proj_path / 'data' / 'pkl_files' / 'bird_train_parsed.pkl', 'wb') as f:
#     pickle.dump(train_parsed, f)

# with open(proj_path / 'data' / 'pkl_files' / 'bird_dev_parsed.pkl', 'wb') as f:
#     pickle.dump(dev_parsed, f)

# with open(proj_path / 'data' / 'pkl_files' / 'bird_test_parsed.pkl', 'wb') as f:
#     pickle.dump(test_parsed, f)

with open(proj_path / 'data' / 'pkl_files' / 'bird_dev_parsed.pkl', 'rb') as f:
    dev_parsed = pickle.load(f)

with open(proj_path / 'data' / 'pkl_files' / 'bird_test_parsed.pkl', 'rb') as f:
    test_parsed = pickle.load(f)

In [ ]:
from itertools import combinations, product
from collections import defaultdict
from src.eval_utils import get_all_partial_score

def measure_inter_score(parsed1: dict[str, tuple], parsed2: dict[str, tuple]):
    results = defaultdict()
    assert len(parsed1) == len(parsed2), f"Length mismatch-1: {len(parsed1)} 2:{len(parsed2)}"
    db_ids = list(parsed1.keys())
    for db_id in db_ids:
        o1 = parsed1[db_id]
        o2 = parsed2[db_id]
        n1 = len(o1)
        n2 = len(o2)
        semantic_sim = np.zeros((n1, n2), dtype=np.float32)
        structural_sim = np.zeros((n1, n2), dtype=np.float32)
        overall_sim = np.zeros((n1, n2), dtype=np.float32)

        idxs = list(product(range(n1), range(n2)))
        iterator = tqdm(idxs, total=len(idxs), desc=f"{db_id}")
        for i, j in iterator:
            ei = o1[i]
            ej = o2[j]

            _, final_score = get_all_partial_score(ei, ej, use_bert=True)

            structural_sim[i, j] = final_score['structural']
            semantic_sim[i, j] = final_score['semantic']
            overall_sim[i, j] = final_score['overall']

        results[db_id] = {
            'semantic': semantic_sim,
            'struct': structural_sim,
            'overall': overall_sim
        }
    return results

results = measure_inter_score(dev_parsed, test_parsed)
with (proj_path / 'data' / 'pkl_files' / 'bird_dev_test_similarity.pkl').open('wb') as f:
    pickle.dump(results, f)

# Complexity between datasets

In [ ]:
def measure_complexity(samples, tables):
    cs = []
    for s in tqdm(samples, total=len(samples)):
        schema = Schema(tables[s.db_id].db_schema)
        output = extract_all(s.final.sql, schema)
        complexity = get_complexity(output)
        cs.append(complexity)
    return cs

train_complexities = measure_complexity(train_samples, bird_tables)
dev_complexities = measure_complexity(dev_samples, bird_tables)
test_complexities = measure_complexity(test_samples, bird_tables)

In [ ]:
for c, n in zip([train_complexities, dev_complexities, test_complexities], ['train', 'dev  ', 'test ']):
    print(f'[{n}] Mean={np.mean(c):.4f} +/-{np.std(c):.4f}, Median={np.median(c):.4f}')

In [ ]:
stats = defaultdict(list)
for s in dev_samples:
    stats[s.db_id].append(s)